# Saws Capstone Project

## Introduction

The San Antonio Water System (SAWS) has collected a dataset of over 3100 Sanitary Sewer Overflow (SSO) events going back to 2009. The data on these events contains a wealth of information about the event, including the root cause that was determined. We know that some of the events are weather related so we will be using data collected by National Oceanic and Atomospheric Administration (NOAA) in combination with the SSO events to provide better insights into what causes these events. It is our hope that these insights can be used by the city of San Antonio and SAWS to plan preventative measures that will benefit the city in the future.

## Imports

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import classification_report
from sklearn import tree

import graphviz
from graphviz import Graph

import acquire
import prepare
import preprocessing

## Acquire

### Acquisition of SSO Data

### Acquisition of Weather Data

The weather data is collected from the National Centers for Environmental Information API. Documentation for the API can be found here: [NCEI API](https://www.ncei.noaa.gov/support/access-data-service-api-user-documentation)
The API request can be found in the acquire file but the details are as follows:
* The data is from the San Antonio International Airport
* The data is the daily summary from December 3, 2008 to April 3, 2019
    * While this is outside the range of the SSO events data, we have done this so that we can take rolling averages up to a month before the first event.
* The data includes information on precipitation, wind, temperature, and a vast number of meteorological events.
* The units for these recording are in standard format.

In [2]:
weather = acquire.acquire_weather()
weather

,STATION,DATE,AWND,FMTM,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT03,WT05,WT06,WT07,WT08,WT09,WT10,WT11,WT13,WT16,WT17,WT18,WT19
0,USW00012921,2008-12-03,9.40,2302.0,2315.0,0.0,0.0,0.0,NaN,77,48,360,350.0,19.9,38.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
1,USW00012921,2008-12-04,11.41,828.0,506.0,0.0,0.0,0.0,NaN,61,39,20,350.0,21.0,29.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USW00012921,2008-12-05,6.71,952.0,839.0,0.0,0.0,0.0,NaN,55,36,20,30.0,15.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USW00012921,2008-12-06,3.80,1157.0,1132.0,0.0,0.0,0.0,NaN,69,30,240,270.0,10.1,14.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USW00012921,2008-12-07,6.93,1557.0,1512.0,0.0,0.0,0.0,NaN,71,36,160,190.0,17.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3769,USW00012921,2019-03-30,13.87,NaN,NaN,0.0,0.0,0.0,67.0,70,51,10,20.0,32.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3770,USW00012921,2019-03-31,10.96,NaN,NaN,0.0,0.0,0.0,52.0,59,44,20,30.0,25.1,35.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3771,USW00012921,2019-04-01,5.59,NaN,NaN,0.0,0.0,0.0,51.0,62,42,130,130.0,12.1,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3772,USW00012921,2019-04-02,8.05,NaN,NaN,0.0,0.0,0.0,53.0,71,37,150,160.0,18.1,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Preparation

### Preparation of SSO Data

### Preparation of Weather Data

* What are we changing?
    * We are removing columns that are don't provide a lot of information.
        * "STATION","AWND", "FMTM", "PGTM", "WDF2", "WDF5", "WSF2", "WSF5", "WT06", "WT07", "WT08", "WT10", "WT11", "WT13", "WT17", "WT18", "WT19"
    * We changed the column names to make them more understandable.
    * We imputed some missing values
        * avg_temp was missing some values so we imputed with `(max_temp + min_temp ) / 2`
        * All of the weather event columns are missing the 0 for if the event didn't happen that day so we replace NaNs with 0.
    * We added rolling averages for precipitation, avg_temp, max_temp, and min_temp
        * These were added at 7, 14, and 30 day periods

In [3]:
prepare.prep_weather_data()

,precipitation,snowfall,snow_depth,avg_temp,max_temp,min_temp,foggy,heavy_fog,thunder,hail,blowing_snow,rain,precipitation_rolling_7,precipitation_rolling_14,precipitation_rolling_30,avg_temp_rolling_7,avg_temp_rolling_14,avg_temp_rolling_30,max_temp_rolling_7,max_temp_rolling_14,max_temp_rolling_30,min_temp_rolling_7,min_temp_rolling_14,min_temp_rolling_30
date,,,,,,,,,,,,,,,,,,,,,,,,
2008-12-03,0.0,0.0,0.0,62.5,77,48,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-12-04,0.0,0.0,0.0,50.0,61,39,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-12-05,0.0,0.0,0.0,45.5,55,36,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-12-06,0.0,0.0,0.0,49.5,69,30,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-12-07,0.0,0.0,0.0,53.5,71,36,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-03-30,0.0,0.0,0.0,67.0,70,51,0,0,0,0,0,0,1.428571e-03,0.019286,0.015333,68.000000,64.142857,60.733333,78.571429,75.142857,70.600000,58.428571,54.142857,51.266667
2019-03-31,0.0,0.0,0.0,52.0,59,44,0,0,0,0,0,0,1.428571e-03,0.019286,0.015000,65.571429,64.071429,60.833333,75.428571,74.785714,70.633333,55.428571,54.285714,51.200000
2019-04-01,0.0,0.0,0.0,51.0,62,42,0,0,0,0,0,0,1.103532e-15,0.019286,0.015000,62.428571,63.785714,60.666667,72.000000,74.357143,70.666667,52.428571,54.142857,50.833333


### Joining SSO and Weather Data

After we have prepared both datasets we are now able to join the SSO events on to the weather data by date.

In [4]:
df = prepare.get_data()
df

,sso_id,report_date,spill_address_num,spill_st_name,total_gal,gals_ret,spill_start,spill_stop,hrs,cause,comments,actions,watershed,unit_id,unit_id2,discharge_to,discharge_route,council_district,month,year,week,earz_zone,pipe_diam,pipe_len,pipe_type,inst_year,inches_no,rainfall_last3,spill_address_full,num_spills_recorded,num_spills_24mos,prevspill_24mos,unit_type,asset_type,last_cleaned,response_time,response_dttm,public_notice,root_cause,hrs_2,gal_2,hrs_3,gal_3,days_since_cleaned,total_gal_binned,country_address,location,zip_code,age,age_binned,time_spilled,precipitation,snowfall,snow_depth,avg_temp,max_temp,min_temp,foggy,heavy_fog,thunder,hail,blowing_snow,rain,precipitation_rolling_7,precipitation_rolling_14,precipitation_rolling_30,avg_temp_rolling_7,avg_temp_rolling_14,avg_temp_rolling_30,max_temp_rolling_7,max_temp_rolling_14,max_temp_rolling_30,min_temp_rolling_7,min_temp_rolling_14,min_temp_rolling_30
0,6582,2019-03-10,3200,THOUSAND OAKS DR,2100,2100.0,2019-03-10 13:16:00,2019-03-10 14:40:00,1.400000,Grease,Spill ContainedReturned to SystemArea Cleaned ...,CLEANED MAIN,SALADO CREEK,66918,66917,STREET,None,NaN,3,2019,11,0,8.0,16.550000,PVC,1997.0,NaN,NaN,3200 THOUSAND OAKS DR,1,1.0,0,GRAVITY,Sewer Main,NaN,27.0,2019-03-10 13:43:00,False,NaN,0.00,0.0,0.0,0.0,NaN,"(1000, 5000]","3200 THOUSAND OAKS DR,SAN ANTONIO,Texas,USA","3200, Thousand Oaks Drive, Horseshoe Bend, Los...",78746,22.0,"(20, 25]",01:24:00,0.00,0.0,0.0,69.0,79,60,1,1,0,0,0,0,4.285714e-03,0.005714,0.016333,54.285714,55.285714,54.200000,65.857143,65.714286,64.533333,46.000000,47.642857,45.900000
1,6583,2019-03-10,6804,S FLORES ST,80,0.0,2019-03-10 14:25:00,2019-03-10 15:45:00,1.333333,Grease,Spill ContainedArea Cleaned and Disinfected,CLEANED MAIN,DOS RIOS,24250,24193,STORMDRAIN,None,3.0,3,2019,11,0,8.0,157.000000,PVC,1988.0,NaN,NaN,6804 S FLORES,1,1.0,0,GRAVITY,Sewer Main,NaN,65.0,2019-03-10 15:30:00,False,NaN,0.00,0.0,0.0,0.0,NaN,"(50, 250]","6804 S FLORES,SAN ANTONIO,Texas,USA","Flores, West Odessa, Ector County, Texas, Unit...",Texas,31.0,"(30, 35]",01:20:00,0.00,0.0,0.0,69.0,79,60,1,1,0,0,0,0,4.285714e-03,0.005714,0.016333,54.285714,55.285714,54.200000,65.857143,65.714286,64.533333,46.000000,47.642857,45.900000
2,6581,2019-03-09,215,AUDREY ALENE DR,79,0.0,2019-03-09 18:00:00,2019-03-09 19:30:00,1.500000,Structural,Spill ContainedArea Cleaned and DisinfectedFlu...,CLEANED MAIN,DOS RIOS,2822,3351,ALLEY,None,1.0,3,2019,10,0,8.0,350.000000,CP,1955.0,NaN,NaN,215 Audrey Alene Dr,1,1.0,0,GRAVITY,Sewer Main,NaN,60.0,2019-03-09 19:00:00,False,NaN,1.15,69.0,0.0,0.0,NaN,"(50, 250]","215 Audrey Alene Dr,SAN ANTONIO,Texas,USA","215, Audrey Alene Drive, San Antonio, Bexar Co...",78216,64.0,"(60, 65]",01:30:00,0.03,0.0,0.0,74.0,87,60,1,0,0,0,0,0,4.285714e-03,0.005714,0.016333,51.714286,54.357143,53.300000,62.142857,64.785714,63.333333,42.571429,46.714286,45.133333
3,6584,2019-03-09,3602,SE MILITARY DR,83,0.0,2019-03-09 15:37:00,2019-03-09 17:00:00,1.383333,Grease,Spill ContainedArea Cleaned and DisinfectedFlu...,NaN,SALADO CREEK,92804,92805,EASEMENT,None,3.0,3,2019,10,0,8.0,213.910000,PVC,1983.0,NaN,NaN,3602 SE MILITARY DR,1,1.0,0,GRAVITY,Sewer Main,NaN,33.0,2019-03-09 16:10:00,False,NaN,0.00,0.0,0.0,0.0,NaN,"(50, 250]","3602 SE MILITARY DR,SAN ANTONIO,Texas,USA","Quality Suites, Southeast Military Drive, Hill...",78223,36.0,"(35, 40]",01:23:00,0.03,0.0,0.0,74.0,87,60,1,0,0,0,0,0,4.285714e-03,0.005714,0.016333,51.714286,54.357143,53.300000,62.142857,64.785714,63.333333,42.571429,46.714286,45.133333
4,6580,2019-03-06,100,PANSY LN,75,0.0,2019-03-06 09:40:00,2019-03-06 09:55:00,0.250000,Structural,Spill ContainedArea Cleaned and DisinfectedFlu...,CLEANED MAIN,SALADO CREEK,61141,49543,STREET,None,2.0,3,2019,10,0,12.0,291.900000,CP,1952.0,NaN,NaN,100 PANSY LN,2,2.0,2018-12-15 00:00:00,GRAVITY,Sewer Main,NaN,0.0,2019-03-06 09:40:00,False,NaN,0.00,0.0,0.0,0.0,NaN,"(50, 250]","100 PANSY LN,SAN ANTONIO,Texas,USA","Pansy Lane, San Antonio, Bexar County, Texas, ...",78209,67.0,"

## Exploration

### Does the age of the sewer drive certain root causes? 

### Does the amount of rain drive certain root causes?

### Does whether or not it rained drive certain root causes?

### Does the temperature drive certain root causes?

## Modeling

### Prepocessing

* What are we changing?
    * We are removing several columns that we believe will not help the model
        * 'sso_id', 'report_date', 'spill_address_num', 'spill_st_name', 'spill_stop', 'spill_start', 'cause', 'comments', 'actions', 'month', 'year', 'week', 'spill_address_full', 'last_cleaned', 'response_dttm', 'prevspill_24mos', 'public_notice', 'country_address', 'location', 'inches_no', 'rainfall_last3', 'unit_id', 'unit_id2', 'zip_code', 'discharge_to', 'discharge_route', 'council_district', 'time_spilled', 'hrs', 'gals_ret', 'pipe_diam', 'pipe_len', 'response_time', 'days_since_cleaned', 'age'
    * We are one hot encoding the categorical columns
        * "watershed", "earz_zone", "pipe_type", "inst_year", "unit_type", "asset_type", "age_binned", "total_gal_binned"

In [5]:
df = preprocessing.get_model_data()

### Decision Tree Model

#### Let's split the data

In [6]:
X = df.drop(columns='root_cause')
y = df.root_cause

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=13)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = .25, random_state=13)

#### Now let's make the model. Through testing we found the the best depth for the tree is 5.

In [7]:
tree = DecisionTreeClassifier(max_depth=5, random_state=13)
tree.fit(X_train, y_train)

print(f'Accuracy Score on Train: {tree.score(X_train, y_train) * 100:.2f}%')
print(f'Accuracy Score on Val: {tree.score(X_val, y_val) * 100:.2f}%')

Accuracy Score on Train: 51.34%
Accuracy Score on Val: 49.76%


#### We can see that our model doesn't appear to be overfit.
#### Let's look at the classification report for the validation set.

In [8]:
preds = tree.predict(X_val)
print(classification_report(y_val, preds))

                   precision    recall  f1-score   support

          Unknown       0.00      0.00      0.00         1
by pass pump leak       0.00      0.00      0.00         1
       contractor       0.00      0.00      0.00        36
           debris       0.20      0.01      0.02        83
           grease       1.00      0.02      0.03       122
              i/i       0.00      0.00      0.00         5
     lift station       0.80      0.25      0.38        32
            other       0.00      0.00      0.00        15
       rain event       0.74      0.83      0.78        81
            roots       0.00      0.00      0.00         5
       structural       0.45      0.97      0.62       246
        vandalism       0.00      0.00      0.00        10

         accuracy                           0.50       637
        macro avg       0.27      0.17      0.15       637
     weighted avg       0.53      0.50      0.37       637



/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### The code below allows for us to look at how the tree is making decision.

In [9]:
# dot_data = export_graphviz(tree, 
#                            feature_names= X_train.columns.tolist(),
#                            class_names= y_train.value_counts(normalize=True).index.sort_values().tolist(),
#                            out_file=None) 
# graph = graphviz.Source(dot_data) 

# graph.render('saws-prediction-tree', view=True)